In [1]:
#import libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.layers.convolutional import Conv2D
from keras.layers import Dense, Conv2D ,Flatten,Dropout,MaxPool2D, BatchNormalization
from keras.utils import np_utils
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory  
from tensorflow.keras.preprocessing import image
import keras
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np 
import tensorflow as tf
import pandas as pd
import seaborn
from sklearn.metrics import confusion_matrix , classification_report
from tensorflow.keras.applications  import DenseNet121
from tensorflow.keras import layers

In [2]:
#import our data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,rotation_range=10, fill_mode='nearest',
     featurewise_center=True,
    featurewise_std_normalization=True,
    vertical_flip= True,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range = (0.4,0.6),
    horizontal_flip=True,validation_split=0.3)

In [3]:
train_generator = train_datagen.flow_from_directory(
    '../input/arabic-letters-numbers-ocr/Dataset',
    target_size=(224, 224),
    batch_size=32,subset='training'
  )

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../input/arabic-letters-numbers-ocr/Dataset'

In [4]:
test_generator = train_datagen.flow_from_directory(
     '../input/arabic-letters-numbers-ocr/Dataset',
    target_size=(224, 224),
    batch_size=32,subset='validation'
  )

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../input/arabic-letters-numbers-ocr/Dataset'

In [ ]:
train_dataset = image_dataset_from_directory('../input/arabic-letters-numbers-ocr/Dataset',
                                             shuffle=True,
                                             batch_size=32,image_size=(224,224))
class_name =  train_dataset.class_names
plt.figure(figsize = (10,10))
for image , label in train_dataset.take(1):
    for i in range(9) :
        plt.subplot(3,3,i+1)
        plt.imshow(image[i].numpy().astype("uint8"))
        plt.title(class_name[label[i]])
        plt.axis("off")

In [ ]:
#call Dense model
dense_model = DenseNet121(include_top=True , weights='imagenet')
for models in dense_model.layers:
    models.trainable= False

In [ ]:
#removing the last 2 alyer to get rid of output layer in DenseNet121
dense_model = keras.Model(inputs=dense_model.input, outputs=dense_model.layers[-2].output)
model = tf.keras.Sequential()
# for layer in dense_model.layers:
model.add(dense_model)
model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dense(39, activation='softmax'))

In [ ]:
model.compile(optimizer="adam", loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', save_best_only=True, monitor='val_loss', mode='min')


In [ ]:
model.summary()

In [ ]:
FIT =model.fit(train_generator,
    validation_data = test_generator, 
    callbacks = [early,checkpoint],
    epochs = 20)

In [ ]:
#plotting training values
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

acc = FIT.history['accuracy']
val_acc = FIT.history['val_accuracy']
loss = FIT.history['loss']
val_loss = FIT.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:

#predict test data
y = model.predict(test_generator)
y =  np.argmax(y,axis= 1 )




In [ ]:
#y_true and y_pred
y_true = np.array([])
y_pred = np.array([])
i = 0
for data, labels in test_generator:
    i += 1
    y = np.argmax(model.predict(data), axis=1)
    y_true = np.append(y_true, labels)
    y_pred = np.append(y_pred, y)
    
    if i == test_generator.samples // 32 + 1:
        break


In [ ]:
model.evaluate(test_generator)

In [ ]:
class_names = {}
l = list(test_generator.class_indices.keys())
for i in range(len(l)) :
    class_names[i] = l[i]

In [ ]:
#example 1
from keras.preprocessing import image
import numpy as np
image_path = "../input/arabic-letters-numbers-ocr/Dataset/ا/1023.png"
new_img = image.load_img(image_path, target_size=(64, 64))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
prediction = model.predict(img)
prediction = np.argmax(prediction,axis=1)
print(prediction)
print(class_names[prediction[0]])
plt.imshow(new_img)

In [ ]:
cm  = confusion_matrix(y_pred,y_true)
plt.figure(figsize=(32,32))
df_cm = pd.DataFrame(cm, index = [i for i in range(39)],
                  columns = [i for i in range(39)])
seaborn.heatmap(df_cm, annot=True, annot_kws={"size": 10}, fmt='d')
plt.title('confusion matrix')
plt.xlabel('prediction')
plt.ylabel('Actual');
plt.show()
